# Data Documentation: Home Credit Default Risk Dataset

This notebook provides comprehensive documentation of all data files used in the Credit Risk Alternative Data project.

---

## Table of Contents

1. [Dataset Overview](#1-dataset-overview)
2. [application_train.csv - Main Application Data](#2-application_traincsv)
3. [bureau.csv - Bureau Credit History](#3-bureaucsv)
4. [bureau_balance.csv - Bureau Monthly Snapshots](#4-bureau_balancecsv)
5. [previous_application.csv - Prior Applications](#5-previous_applicationcsv)
6. [credit_card_balance.csv - Credit Card Data](#6-credit_card_balancecsv)
7. [POS_CASH_balance.csv - POS/Cash Loans](#7-pos_cash_balancecsv)
8. [installments_payments.csv - Payment History](#8-installments_paymentscsv)
9. [Data Relationships](#9-data-relationships)
10. [Feature Engineering](#10-feature-engineering)

---

# 1. Dataset Overview

## Source

The data comes from the **Home Credit Default Risk** competition on Kaggle. Home Credit is an international consumer finance provider that serves populations with little or no credit history.

## Business Context

Home Credit wants to predict whether a client will repay a loan or default. The challenge is that many of their customers have little or no credit history, making traditional credit scoring methods inadequate.

## Data Statistics

| File | Records | Columns | Size | Key Fields |
|------|---------|---------|------|------------|
| application_train.csv | 307,511 | 122 | 158 MB | SK_ID_CURR, TARGET |
| application_test.csv | 48,744 | 121 | 25 MB | SK_ID_CURR (no TARGET) |
| bureau.csv | 1,716,428 | 17 | 162 MB | SK_ID_CURR, SK_ID_BUREAU |
| bureau_balance.csv | 27,299,925 | 3 | 358 MB | SK_ID_BUREAU, MONTHS_BALANCE |
| previous_application.csv | 1,670,214 | 37 | 386 MB | SK_ID_CURR, SK_ID_PREV |
| credit_card_balance.csv | 3,840,312 | 23 | 405 MB | SK_ID_CURR, SK_ID_PREV |
| POS_CASH_balance.csv | 10,001,358 | 8 | 375 MB | SK_ID_CURR, SK_ID_PREV |
| installments_payments.csv | 13,605,401 | 8 | 690 MB | SK_ID_CURR, SK_ID_PREV |

**Total:** ~58 million records, ~2.6 GB

---

# 2. application_train.csv

## Description

The main application data containing information about each loan application. This is the **primary table** that all other tables join to.

## Key Fields

| Field | Type | Description | Example |
|-------|------|-------------|--------|
| **SK_ID_CURR** | int | Unique customer ID | 100001 |
| **TARGET** | int | Default indicator (0=repaid, 1=default) | 0 or 1 |

## Demographic Features

| Field | Type | Description | Range/Values |
|-------|------|-------------|-------------|
| CODE_GENDER | str | Gender | M/F/XNA |
| FLAG_OWN_CAR | str | Owns a car | Y/N |
| FLAG_OWN_REALTY | str | Owns real estate | Y/N |
| CNT_CHILDREN | int | Number of children | 0-19 |
| AMT_INCOME_TOTAL | float | Total income | 25,650 - 117M |
| NAME_INCOME_TYPE | str | Income source | Working, Commercial associate, etc. |
| NAME_EDUCATION_TYPE | str | Education level | Secondary, Higher education, etc. |
| NAME_FAMILY_STATUS | str | Marital status | Married, Single, etc. |
| NAME_HOUSING_TYPE | str | Housing situation | House/apartment, With parents, etc. |

## Loan Features

| Field | Type | Description | Range |
|-------|------|-------------|-------|
| NAME_CONTRACT_TYPE | str | Loan type | Cash loans, Revolving loans |
| AMT_CREDIT | float | Loan amount | 45,000 - 4,050,000 |
| AMT_ANNUITY | float | Loan annuity | 1,615 - 258,025 |
| AMT_GOODS_PRICE | float | Price of goods | 40,500 - 4,050,000 |

## Time Features (in negative days from application)

| Field | Type | Description | Interpretation |
|-------|------|-------------|---------------|
| DAYS_BIRTH | int | Age in days | -7,489 = 20.5 years old |
| DAYS_EMPLOYED | int | Employment duration | -365 = 1 year employed |
| DAYS_REGISTRATION | int | Registration date | Days since ID registration |
| DAYS_ID_PUBLISH | int | ID publish date | Days since ID document issued |

## External Sources (Alternative Data)

| Field | Type | Description | Range |
|-------|------|-------------|-------|
| EXT_SOURCE_1 | float | External score 1 | 0-1 (normalized) |
| EXT_SOURCE_2 | float | External score 2 | 0-1 (normalized) |
| EXT_SOURCE_3 | float | External score 3 | 0-1 (normalized) |

**Note:** These are the most predictive features in the dataset!

## Document Flags (Alternative Data)

| Field | Type | Description |
|-------|------|-------------|
| FLAG_DOCUMENT_2 to FLAG_DOCUMENT_21 | int | Document provided (0/1) |

## Contact Flags (Alternative Data)

| Field | Type | Description |
|-------|------|-------------|
| FLAG_MOBIL | int | Mobile phone provided |
| FLAG_EMP_PHONE | int | Work phone provided |
| FLAG_WORK_PHONE | int | Work phone (home) provided |
| FLAG_CONT_MOBILE | int | Mobile reachable |
| FLAG_PHONE | int | Home phone provided |
| FLAG_EMAIL | int | Email provided |

---

# 3. bureau.csv

## Description

Data about client's previous credits from other financial institutions reported to the Credit Bureau. Each row represents one credit from the bureau.

## Key Fields

| Field | Type | Description |
|-------|------|-------------|
| **SK_ID_CURR** | int | Customer ID (joins to application) |
| **SK_ID_BUREAU** | int | Bureau credit ID (joins to bureau_balance) |

## Credit Information

| Field | Type | Description |
|-------|------|-------------|
| CREDIT_ACTIVE | str | Status: Active, Closed, Sold, Bad debt |
| CREDIT_CURRENCY | str | Currency of credit |
| DAYS_CREDIT | int | Days before application that bureau credit was issued |
| CREDIT_DAY_OVERDUE | int | Days past due at time of application |
| DAYS_CREDIT_ENDDATE | int | Days until credit ends |
| DAYS_ENDDATE_FACT | int | Actual end date (if closed) |
| AMT_CREDIT_MAX_OVERDUE | float | Maximum overdue amount |
| CNT_CREDIT_PROLONG | int | Times credit was prolonged |
| AMT_CREDIT_SUM | float | Current credit amount |
| AMT_CREDIT_SUM_DEBT | float | Current debt amount |
| AMT_CREDIT_SUM_LIMIT | float | Credit limit |
| AMT_CREDIT_SUM_OVERDUE | float | Current overdue amount |
| CREDIT_TYPE | str | Type of credit (Consumer, Car, Mortgage, etc.) |
| DAYS_CREDIT_UPDATE | int | Days since last update |
| AMT_ANNUITY | float | Annuity amount |

## Aggregation Strategy

```python
# Aggregate to customer level using:
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max', 'min'],
    'DAYS_CREDIT': ['mean', 'max', 'min'],
    # ... for all numeric columns
})
```

**Result:** One row per customer with BUREAU_* prefixed features

---

# 4. bureau_balance.csv

## Description

Monthly balance snapshots for each bureau credit. Provides time-series view of credit behavior.

## Fields

| Field | Type | Description | Values |
|-------|------|-------------|--------|
| **SK_ID_BUREAU** | int | Bureau credit ID | Links to bureau.csv |
| MONTHS_BALANCE | int | Month relative to application | -96 to 0 (0 = current) |
| STATUS | str | Payment status | 0,1,2,3,4,5,C,X |

## STATUS Values

| Status | Meaning |
|--------|--------|
| 0 | Days past due (DPD) 0-29 (good) |
| 1 | DPD 30-59 |
| 2 | DPD 60-89 |
| 3 | DPD 90-119 |
| 4 | DPD 120-149 |
| 5 | DPD 150+ (very bad) |
| C | Closed (paid off) |
| X | Unknown status |

## Aggregation Strategy

```python
# First: aggregate to bureau level
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
    'MONTHS_BALANCE': ['min', 'max', 'mean'],
    'STATUS': lambda x: (x == '0').sum()  # Count of good months
})

# Then: merge with bureau and aggregate to customer level
```

---

# 5. previous_application.csv

## Description

All previous applications for Home Credit loans (not bureau data). Shows application history at Home Credit.

## Key Fields

| Field | Type | Description |
|-------|------|-------------|
| **SK_ID_CURR** | int | Customer ID |
| **SK_ID_PREV** | int | Previous application ID |

## Application Information

| Field | Type | Description |
|-------|------|-------------|
| NAME_CONTRACT_TYPE | str | Type: Cash, Revolving, Consumer |
| AMT_ANNUITY | float | Annuity of previous loan |
| AMT_APPLICATION | float | Amount requested |
| AMT_CREDIT | float | Amount approved |
| AMT_DOWN_PAYMENT | float | Down payment |
| AMT_GOODS_PRICE | float | Goods price |
| NAME_CONTRACT_STATUS | str | Approved, Refused, Canceled, Unused offer |
| DAYS_DECISION | int | Days before current application when decision was made |
| NAME_PAYMENT_TYPE | str | Payment method |
| CODE_REJECT_REASON | str | Reason for rejection (if applicable) |
| NAME_CLIENT_TYPE | str | New, Refreshed, Repeater |
| NAME_GOODS_CATEGORY | str | Category of goods |
| NAME_PORTFOLIO | str | Type of portfolio |
| NAME_PRODUCT_TYPE | str | Product type |
| CHANNEL_TYPE | str | Sales channel |
| SELLERPLACE_AREA | int | Seller place area |
| NAME_SELLER_INDUSTRY | str | Seller industry |
| CNT_PAYMENT | float | Number of payments |
| NAME_YIELD_GROUP | str | Interest rate group |
| PRODUCT_COMBINATION | str | Product detail |

## Aggregation Strategy

```python
prev_agg = previous_application.groupby('SK_ID_CURR').agg({
    col: ['mean', 'max', 'min'] for col in numeric_columns
})
# Result: PREV_* prefixed features
```

---

# 6. credit_card_balance.csv

## Description

Monthly balance snapshots for previous credit cards at Home Credit.

## Key Fields

| Field | Type | Description |
|-------|------|-------------|
| **SK_ID_CURR** | int | Customer ID |
| **SK_ID_PREV** | int | Previous loan ID |
| MONTHS_BALANCE | int | Month relative to application |

## Balance Information

| Field | Type | Description |
|-------|------|-------------|
| AMT_BALANCE | float | Balance at statement date |
| AMT_CREDIT_LIMIT_ACTUAL | int | Credit limit |
| AMT_DRAWINGS_ATM_CURRENT | float | ATM withdrawals this month |
| AMT_DRAWINGS_CURRENT | float | Total drawings this month |
| AMT_DRAWINGS_OTHER_CURRENT | float | Other drawings |
| AMT_DRAWINGS_POS_CURRENT | float | POS drawings |
| AMT_INST_MIN_REGULARITY | float | Minimum installment |
| AMT_PAYMENT_CURRENT | float | Payment this month |
| AMT_PAYMENT_TOTAL_CURRENT | float | Total payment |
| AMT_RECEIVABLE_PRINCIPAL | float | Receivable principal |
| AMT_RECIVABLE | float | Total receivable |
| AMT_TOTAL_RECEIVABLE | float | Total receivable (including fees) |
| CNT_DRAWINGS_ATM_CURRENT | int | Count of ATM withdrawals |
| CNT_DRAWINGS_CURRENT | int | Count of all drawings |
| CNT_DRAWINGS_OTHER_CURRENT | int | Count of other drawings |
| CNT_DRAWINGS_POS_CURRENT | int | Count of POS drawings |
| CNT_INSTALMENT_MATURE_CUM | int | Cumulative matured installments |
| NAME_CONTRACT_STATUS | str | Contract status |
| SK_DPD | int | Days past due |
| SK_DPD_DEF | int | Days past due (tolerated) |

## Aggregation Strategy

```python
cc_agg = credit_card_balance.groupby('SK_ID_CURR').agg({
    col: ['mean', 'max', 'min'] for col in numeric_columns
})
# Result: CC_* prefixed features
```

---

# 7. POS_CASH_balance.csv

## Description

Monthly balance snapshots for previous POS (point of sales) and cash loans at Home Credit.

## Fields

| Field | Type | Description |
|-------|------|-------------|
| **SK_ID_CURR** | int | Customer ID |
| **SK_ID_PREV** | int | Previous loan ID |
| MONTHS_BALANCE | int | Month relative to application |
| CNT_INSTALMENT | int | Total installments |
| CNT_INSTALMENT_FUTURE | int | Remaining installments |
| NAME_CONTRACT_STATUS | str | Contract status |
| SK_DPD | int | Days past due |
| SK_DPD_DEF | int | Days past due (tolerated) |

## Aggregation Strategy

```python
pos_agg = POS_CASH_balance.groupby('SK_ID_CURR').agg({
    col: ['mean', 'max', 'min'] for col in numeric_columns
})
# Result: POS_* prefixed features
```

---

# 8. installments_payments.csv

## Description

Payment history for previous loans at Home Credit. Each row shows when a payment was due and when it was actually paid.

## Fields

| Field | Type | Description |
|-------|------|-------------|
| **SK_ID_CURR** | int | Customer ID |
| **SK_ID_PREV** | int | Previous loan ID |
| NUM_INSTALMENT_VERSION | float | Version of installment schedule |
| NUM_INSTALMENT_NUMBER | int | Installment number |
| DAYS_INSTALMENT | int | When payment was due (relative to application) |
| DAYS_ENTRY_PAYMENT | int | When payment was actually made |
| AMT_INSTALMENT | float | Amount that was due |
| AMT_PAYMENT | float | Amount that was paid |

## Derived Features

```python
# Payment behavior features
installments['PAYMENT_DIFF'] = installments['AMT_PAYMENT'] - installments['AMT_INSTALMENT']
installments['PAYMENT_RATIO'] = installments['AMT_PAYMENT'] / (installments['AMT_INSTALMENT'] + 0.0001)

# PAYMENT_DIFF > 0: Overpaid (good customer)
# PAYMENT_DIFF < 0: Underpaid (risk signal)
# PAYMENT_RATIO < 1: Underpaid
# PAYMENT_RATIO > 1: Overpaid
```

## Aggregation Strategy

```python
inst_agg = installments_payments.groupby('SK_ID_CURR').agg({
    col: ['mean', 'max', 'min'] for col in numeric_columns
})
# Result: INST_* prefixed features
```

---

# 9. Data Relationships

## Entity Relationship Diagram

```
+-------------------+
| application_train |  <-- Main table (307,511 customers)
| SK_ID_CURR (PK)   |
| TARGET            |
+--------+----------+
         |
         | 1:N
         |
+--------v----------+        +-------------------+
| bureau            | 1:N    | bureau_balance    |
| SK_ID_CURR (FK)   +------->| SK_ID_BUREAU (FK) |
| SK_ID_BUREAU (PK) |        | MONTHS_BALANCE    |
+-------------------+        +-------------------+
         |
+--------v------------+
| previous_application|  <-- Previous Home Credit applications
| SK_ID_CURR (FK)     |
| SK_ID_PREV (PK)     |
+--------+------------+
         |
         | 1:N to each:
         |
         +------> credit_card_balance (SK_ID_PREV)
         +------> POS_CASH_balance (SK_ID_PREV)
         +------> installments_payments (SK_ID_PREV)
```

## Join Strategy

All tables are **aggregated to the customer level** (SK_ID_CURR) before joining to the main application table:

1. **bureau_balance** -> aggregate to SK_ID_BUREAU
2. **bureau** (with bureau_balance) -> aggregate to SK_ID_CURR
3. **credit_card_balance** -> aggregate to SK_ID_CURR
4. **POS_CASH_balance** -> aggregate to SK_ID_CURR
5. **installments_payments** -> aggregate to SK_ID_CURR
6. **previous_application** -> aggregate to SK_ID_CURR
7. **All aggregates** -> LEFT JOIN to application_train on SK_ID_CURR

---

# 10. Feature Engineering

## 10.1 Engineered Features from application_train

| Feature | Formula | Rationale |
|---------|---------|----------|
| CREDIT_INCOME_RATIO | AMT_CREDIT / AMT_INCOME_TOTAL | Debt burden relative to income |
| ANNUITY_INCOME_RATIO | AMT_ANNUITY / AMT_INCOME_TOTAL | Monthly payment burden |
| CREDIT_GOODS_RATIO | AMT_CREDIT / AMT_GOODS_PRICE | Loan-to-value ratio |
| AGE_YEARS | -DAYS_BIRTH / 365.25 | Human-readable age |
| EMPLOYMENT_YEARS | -DAYS_EMPLOYED / 365.25 | Employment duration |
| DAYS_EMPLOYED_PERCENT | DAYS_EMPLOYED / DAYS_BIRTH | Employment as % of life |
| EXT_SOURCE_MEAN | mean(EXT_SOURCE_1,2,3) | Averaged external scores |
| EXT_SOURCE_STD | std(EXT_SOURCE_1,2,3) | Variability in external scores |
| EXT_SOURCE_MIN | min(EXT_SOURCE_1,2,3) | Worst external score |
| EXT_SOURCE_MAX | max(EXT_SOURCE_1,2,3) | Best external score |

## 10.2 Aggregation Features

For each secondary table, we create aggregates using:

| Aggregation | Meaning | Risk Interpretation |
|-------------|---------|--------------------|
| mean | Average behavior | Typical behavior |
| max | Worst case | Peak risk exposure |
| min | Best case | Minimum risk exposure |

## 10.3 Feature Naming Convention

```
{TABLE}_{COLUMN}_{AGGREGATION}

Examples:
- BUREAU_AMT_CREDIT_SUM_mean
- PREV_AMT_APPLICATION_max
- CC_AMT_BALANCE_min
- POS_SK_DPD_max
- INST_PAYMENT_RATIO_mean
```

## 10.4 Feature Categories

### Traditional Features (334)

Keywords: AMT_, DAYS_, CNT_, CREDIT, INCOME, BUREAU_, PREV_, ANNUITY

These are standard credit bureau features that financial institutions have used for decades.

### Alternative Features (47)

Keywords: FLAG_, EXT_SOURCE, REGION_, OBS_, DEF_, EMAIL, PHONE, MOBIL, SOCIAL

These are non-traditional data sources that can help assess thin-file customers.

## 10.5 Final Feature Count

| Stage | Features |
|-------|----------|
| Raw application | 122 |
| After engineering | ~130 |
| After aggregations | ~400 |
| After encoding | **381** |

The final dataset has 381 features after all preprocessing steps.

---

# Appendix: Data Quality Notes

## Missing Values

| File | Avg Missing % | Strategy |
|------|--------------|----------|
| application | 15% | Fill with 0 |
| bureau | 8% | Fill with 0 |
| Others | <5% | Fill with 0 |

**Note:** Missing values often indicate "no data" rather than truly missing information. For example, missing BUREAU_ features likely mean the customer has no bureau history.

## Outliers

- DAYS_EMPLOYED has value 365243 (1000 years) for unemployed/retired
  - Handled by clipping negative values to 0 after conversion
- AMT_INCOME_TOTAL has extreme outliers (up to 117M)
  - Handled by Yeo-Johnson transformation

## Class Imbalance

| Class | Count | Percentage |
|-------|-------|------------|
| 0 (No default) | 282,686 | 91.93% |
| 1 (Default) | 24,825 | 8.07% |

Handled by SMOTE with 50% sampling ratio.